In [7]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/body-performance-data/bodyPerformance.csv


In [8]:
df = pd.read_csv("/kaggle/input/body-performance-data/bodyPerformance.csv")
df

,age,gender,height_cm,weight_kg,body fat_%,diastolic,systolic,gripForce,sit and bend forward_cm,sit-ups counts,broad jump_cm,class
0,27.0,M,172.3,75.24,21.3,80.0,130.0,54.9,18.4,60.0,217.0,C
1,25.0,M,165.0,55.80,15.7,77.0,126.0,36.4,16.3,53.0,229.0,A
2,31.0,M,179.6,78.00,20.1,92.0,152.0,44.8,12.0,49.0,181.0,C
3,32.0,M,174.5,71.10,18.4,76.0,147.0,41.4,15.2,53.0,219.0,B
4,28.0,M,173.8,67.70,17.1,70.0,127.0,43.5,27.1,45.0,217.0,B
...,...,...,...,...,...,...,...,...,...,...,...,...
13388,25.0,M,172.1,71.80,16.2,74.0,141.0,35.8,17.4,47.0,198.0,C
13389,21.0,M,179.7,63.90,12.1,74.0,128.0,33.0,1.1,48.0,167.0,D
13390,39.0,M,177.2,80.50,20.1,78.0,132.0,63.5,16.4,45.0,229.0,A
13391,64.0,F,146.1,57.70,40.4,68.0,121.0,19.3,9.2,0.0,75.0,D


In [9]:
df.dtypes

age                        float64
gender                      object
height_cm                  float64
weight_kg                  float64
body fat_%                 float64
diastolic                  float64
systolic                   float64
gripForce                  float64
sit and bend forward_cm    float64
sit-ups counts             float64
broad jump_cm              float64
class                       object
dtype: object

In [11]:
X = df.iloc[:, 0:11]
X = pd.get_dummies(X).astype(float)
y = df.iloc[:, 11]

print(X[0:5])
print(y[0:5])

    age  height_cm  weight_kg  body fat_%  diastolic  systolic  gripForce  \
0  27.0      172.3      75.24        21.3       80.0     130.0       54.9   
1  25.0      165.0      55.80        15.7       77.0     126.0       36.4   
2  31.0      179.6      78.00        20.1       92.0     152.0       44.8   
3  32.0      174.5      71.10        18.4       76.0     147.0       41.4   
4  28.0      173.8      67.70        17.1       70.0     127.0       43.5   

   sit and bend forward_cm  sit-ups counts  broad jump_cm  gender_F  gender_M  
0                     18.4            60.0          217.0       0.0       1.0  
1                     16.3            53.0          229.0       0.0       1.0  
2                     12.0            49.0          181.0       0.0       1.0  
3                     15.2            53.0          219.0       0.0       1.0  
4                     27.1            45.0          217.0       0.0       1.0  
0    C
1    A
2    C
3    B
4    B
Name: class, dtype: ob

In [12]:
y = pd.get_dummies(y)

print(y[0:5])

   A  B  C  D
0  0  0  1  0
1  1  0  0  0
2  0  0  1  0
3  0  1  0  0
4  0  1  0  0


In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# 학습과 테스트셋 구분
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)

# 모델 설정
model = Sequential()
model.add(Dense(12, input_dim=12, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(4, activation='softmax'))
model.summary()

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 학습 중단 설정
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=20)

# 저장 폴더와 이름
modelpath = "./data/model/jh-bestmodel.hdf5"

#모델 업데이트하고 저장
checkpointer = ModelCheckpoint(filepath=modelpath, monitor='val_loss', verbose=0, save_best_only=True)

#모델 실행
history = model.fit(X_train, y_train, epochs=1000, batch_size=10, validation_split=0.25, verbose=1, callbacks=[early_stopping_callback, checkpointer])

score = model.evaluate(X_test, y_test)
print('Test accuracy:', score[1])

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_15 (Dense)            (None, 12)                156       
                                                                 
 dense_16 (Dense)            (None, 8)                 104       
                                                                 
 dense_17 (Dense)            (None, 4)                 36        
                                                                 
Total params: 296
Trainable params: 296
Non-trainable params: 0
_________________________________________________________________
Epoch 1/1000
804/804 [==============================] - 3s 3ms/step - loss: 2.0755 - accuracy: 0.2897 - val_loss: 1.3785 - val_accuracy: 0.2945
Epoch 2/1000
804/804 [==============================] - 2s 2ms/step - loss: 1.3668 - accuracy: 0.2951 - val_loss: 1.3566 - val_accuracy: 0.3139
Epoch 3/1000
804/804 [==========